# CNN model training using CIFAR10 dataset

Reference: https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
import os
import time
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

In [ ]:
def train_cnn(batch_size, num_epochs):

    # ---
    # set parameters
    num_classes = 10

    # ---
    # Download and prepare the CIFAR10 dataset
    (train_images, train_labels), (
        test_images,
        test_labels,
    ) = datasets.cifar10.load_data()

    print((train_images.shape, train_labels.shape))
    print((test_images.shape, test_labels.shape))

    # ---
    # Normalize pixel values to be between 0 and 1
    train_images, test_images = train_images / 255.0, test_images / 255.0

    # ---
    # Verify the data
    class_names = [
        "airplane",
        "automobile",
        "bird",
        "cat",
        "deer",
        "dog",
        "frog",
        "horse",
        "ship",
        "truck",
    ]

    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
        # The CIFAR labels happen to be arrays,
        # which is why you need the extra index
        plt.xlabel(class_names[int(train_labels[i][0])])
    plt.savefig("cifar10.png")
    plt.clf()

    # ---
    # Create a CNN model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.summary()
    # Add dense layers on top
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(10))
    model.summary()

    # Compile and train the model
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    #
    start_time = time.time()
    # Training
    history = model.fit(
        train_images,
        train_labels,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(test_images, test_labels),
    )
    #
    end_time = time.time()
    elapsed_time = end_time - start_time
    speed = train_images.shape[0] * num_epochs / elapsed_time
    #
    print("batch size       {:d}".format(batch_size))
    print("Training runtime {:.2f} seconds".format(elapsed_time))
    print("Training speed   {:d} images/sec".format(int(speed)))

    # Evaluate the model
    plt.plot(history.history["accuracy"], label="accuracy")
    plt.plot(history.history["val_accuracy"], label="val_accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.ylim([0.5, 1])
    plt.legend(loc="lower right")
    plt.savefig("train_cnn_cifar10_eval.png")
    plt.clf()

    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

    return elapsed_time, speed, test_acc

In [ ]:
# ---
# set thread numbers (for CPU-only training)
num_threads = 4
os.environ["OMP_NUM_THREADS"] = str(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)

In [ ]:
sum_time = 0.0
sum_speed = 0.0
sum_acc = 0
num_runs = 10

batch_size = 32
num_epochs = 10
for x in range(num_runs):
    elapsed_time, speed, test_acc = train_cnn(batch_size, num_epochs)
    sum_time += elapsed_time
    sum_speed += speed
    sum_acc += test_acc

print("Avg training runtime {:.2f} seconds".format(sum_time / float(num_runs)))
print("Avg training speed   {:d} images/sec".format(int(sum_speed / float(num_runs))))
print("Avg test accuracy    {:.3f} ".format(sum_acc / float(num_runs)))

This notebook was adapted from the code sample of [Tensorflow CNN tutorials](https://www.tensorflow.org/tutorials/images/cnn) which is provided under the [APACHE 2.0 License](https://www.apache.org/licenses/LICENSE-2.0).